## Assignment 2 Data Analysis using Pandas

This assignment will contain 2 question with details as below. The due date is October 23 (Sunday), 2022 23:59PM. Each late day will result in 20% loss of total points.

### Question 1 (20 points) Datacamp courses

Datacamp is the online learning platform for data science courses and modules. As the only collaborator in Portugal, this course provides free access to the students in Business Analytics as the complementary resource. For this assignment, you are expected to complete the following moduels and submit your completion badges:

- [Importing & Cleaning Data with Python](https://app.datacamp.com/learn/skill-tracks/importing-cleaning-data-with-python)
- [Data Manipulation with Python](https://app.datacamp.com/learn/skill-tracks/data-manipulation-with-python)

Each badge will account for 10 points. If you are confident with the knowledge, you may skip the videos and directly finish the challenges. You can submit the badge by the end of the October if you would need more time. 

### Question 2 (80 points) Celluloid ceiling

Wonder Woman             |  Captain Marvel
:-------------------------:|:-------------------------:
![wonderwoman](https://upload.wikimedia.org/wikipedia/en/4/4f/Wonder_Woman_1984_poster.png) | ![marvel](https://upload.wikimedia.org/wikipedia/pt/5/59/Captain_Marvel_%282018%29.jpg)

Women are involved in the film industry in all roles, including as film directors, actresses, cinematographers, film producers, film critics, and other film industry professions, though women have been underrepresented in all these positions. Studies found that women have always had a presence in film acting, but have consistently been underrepresented, and on average significantly less well paid. 

In 2015, Forbes reported that "...just 21 of the 100 top-grossing films of 2014 featured a female lead or co-lead, while only 28.1% of characters in 100 top-grossing films were female... This means it’s much rarer for women to get the sort of blockbuster role which would warrant the massive backend deals many male counterparts demand (Tom Cruise in Mission: Impossible or Robert Downey Jr. in Iron Man, for example)".

Also, Forbes' analysis of US acting salaries in 2013 determined that the "...men on Forbes’ list of top-paid actors for that year made 2½ times as much money as the top-paid actresses. That means that Hollywood's best-compensated actresses made just 40 cents for every dollar that the best-compensated men made. 


In this assignment, we want to examine whether and how women representation is lacking in the film industry. We will adopt The Bechdel test as a measure of the representation of women in the film industry. The test is named after the American cartoonist Alison Bechdel in whose 1985 comic strip Dykes to Watch Out For the test first appeared. **A movie is said to meet the Bechdel test  following three criteria: (1) it has to have at least two women in it, who (2) who talk to each other, about (3) something besides a man.**

We are going to obtain the data ourselves to perform the analysis. Specifically, we will retrieve the movie metadata from IMDB (Internet Movie Database), an online database of information related to films, television programs, home videos, video games, and streaming content online – including cast, production crew and personal biographies, plot summaries, trivia, ratings, and fan and critical reviews. As of January 2020, IMDb has approximately 6.5 million titles (including episodes) and 10.4 million personalities in its database, as well as 83 million registered users.


The IMDb Top 250 is a list of the top rated 250 films, based on ratings by the registered users of the website using the methods described. We will focus on these famous movies in this analysis:

**Question 2.1** (25 points): We will retrieve the metadata of IMDb Top 250 movies from the [IMDb charts](https://www.imdb.com/chart/top/). For each movie on the list, we can scrape the following characteristics from the information page. For example, from the [page of top rated movie "The Shawshank Redemption"](https://www.imdb.com/title/tt0111161/?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=e31d89dd-322d-4646-8962-327b42fe94b1&pf_rd_r=F4QFC0SVZN1HTDHCY3C0&pf_rd_s=center-1&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_tt_1), we want to extract the metadata about this movie as:
- IMDb id (0111161)
- Movie name (The Shawshank Redemption)
- Year (1994)
- Director (Frank Darabont)
- Stars (Tim Robbins, Morgan Freeman, Bob Gunton)
- Rating (9.3)
- Number of reviews (2.6M)
- Genres (Drama)
- Country (USA)
- Language (English)
- Budget (\$25,000,000)
- Gross box Office Revenue (\$28,884,504)

![imdb](https://mrfloris.com/files/images/imdb-top250-page-start.png)


After scraping the 250 movies, save the data as a dataframe ```imdb_top_movies```. 

Also, saving the dataframe to a local file ```imdb_top_movies.csv``` so that later you can load it without scraping the website twice.

Hint: You can get the links to these movies from the IMDb top chart page, and then scrape each movie page by sending the request to these links. At each movie page, the information requested are located at different sections. 

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import re

In [2]:
# request the web page
page = requests.get("https://www.imdb.com/chart/top/")

# parse the HTML document
soup = BeautifulSoup(page.content, 'html.parser')

# create new variable, that stores all instances of the td tag "titleColumn" 
title_urls = soup.find_all(class_="titleColumn")

# loop over all instances, extract the title URL, add it to the "https://www.imdb.com"-path and store it in a list "data"
sub_pages = []
for titles in title_urls:
    find_url = titles.find("a", href = True)
    url = find_url["href"]
    sub_pages.append("https://www.imdb.com{0}".format(url))

# main loop, that scrapes all 250 movie detail pages and extracts the relevant information
data = []
for element in sub_pages:
    a = requests.get(element)
    b = BeautifulSoup(a.content, 'html.parser')
    b.prettify()
# IMDb id
    m = re.search('title/tt(.+?)/', element)
    if m:
        found = m.group(1)
    data.append(found)
# movie name
    title = b.find("h1").get_text()
    data.append(title)
# year
    container = b.find_all(class_="ipc-inline-list__item")[3]
    m = re.search('role="button" tabindex="0">(.+?)</a>', str(container))
    if m:
        year = m.group(1)
    data.append(year)
# director
    container = b.find_all(class_="ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link")
    m = re.search('rel="">(.+?)</a>', str(container))
    if m:
        director = m.group(1)
    data.append(director)
# stars
    container = b.find_all(class_="ipc-inline-list ipc-inline-list--show-dividers ipc-inline-list--inline ipc-metadata-list-item__list-content baseAlt")[2].get_text()
    stars = re.sub(r"(\w)([A-Z])", r"\1, \2", container)
    if "Mc, " in stars:
        stars = stars.replace("Mc, ", "Mc")
    if "Jr." in stars:
        stars = stars.replace("Jr.", "Jr., ")
    if ")" in stars:
        stars = stars.replace(")", "), ")
    if stars[-2] == ",":
        stars = stars[:-2]
    if "(" in stars:
        stars = stars.replace("(", " (")
    if "Di, " in stars:
        stars = stars.replace("Di, ", "Di")
    if "Jr.,  (" in stars:
        stars = stars.replace("Jr.,  (", "Jr. (")
    data.append(stars)
# rating
    container = b.find_all(class_="sc-7ab21ed2-0 fAePGh")
    m = re.search('<span class="sc-7ab21ed2-1 jGRxWM">(.+?)</span><span>/<!-- -->10', str(container))
    if m:
        rating = m.group(1)
    data.append(rating)
# number of reviews
    container = b.find_all(class_="sc-7ab21ed2-3 dPVcnq")
    m = re.search('<div class="sc-7ab21ed2-3 dPVcnq">(.+?)</div>', str(container))
    if m:
        reviews = m.group(1)
    data.append(reviews)
# genre
    container = b.find_all(class_="ipc-chip-list__scroller")[0].get_text()
    genre = re.sub(r"(\w)([A-Z])", r"\1, \2", container)
    data.append(genre)
# country
    container = b.find_all(class_="ipc-inline-list ipc-inline-list--show-dividers ipc-inline-list--inline ipc-metadata-list-item__list-content base")[4].get_text()
    country = re.sub(r"(\w)([A-Z])", r"\1, \2", container)
    data.append(country)
# language
    container = b.find_all(class_="ipc-inline-list ipc-inline-list--show-dividers ipc-inline-list--inline ipc-metadata-list-item__list-content base")[6]  
    if "language" in str(container):
        container = container.get_text()
        language = re.sub(r"(\w)([A-Z])", r"\1, \2", container)
    else:
        container = b.find_all(class_="ipc-inline-list ipc-inline-list--show-dividers ipc-inline-list--inline ipc-metadata-list-item__list-content base")[5].get_text()
        language = re.sub(r"(\w)([A-Z])", r"\1, \2", container)
    data.append(language)
# budget
    try:
        budget = b.find_all("span", class_ = "ipc-metadata-list-item__list-content-item")[2].get_text()
        if "estimated" not in budget:
            budget = float("NAN")
    except:
        budget = float("NAN")
    data.append(budget)
# gross_box_office_revenue
    try:
        gross_box_office_revenue = b.find_all("span", class_ = "ipc-metadata-list-item__list-content-item")[-2].get_text()
        if "$" not in str(gross_box_office_revenue):
            gross_box_office_revenue = b.find_all("span", class_ = "ipc-metadata-list-item__list-content-item")[-1].get_text()
            if "$" not in str(gross_box_office_revenue):
                gross_box_office_revenue = b.find_all("span", class_ = "ipc-metadata-list-item__list-content-item")[-5].get_text()
        if "estimated" in str(gross_box_office_revenue):
            gross_box_office_revenue = float("NAN")
        if "Waterfront" in str(gross_box_office_revenue):
            gross_box_office_revenue = float("NAN")
    except:
        gross_box_office_revenue = float("NAN")
    data.append(gross_box_office_revenue)
# runtime
    container = b.find_all(class_="ipc-metadata-list ipc-metadata-list--dividers-none ipc-metadata-list--compact ipc-metadata-list--base")
    m = re.search('ipc-metadata-list-item__content-container">(.+?)<!-- --> <!-- -->hour', str(container))
    if m:
        hour = m.group(1)
    n = re.search('<!-- --> <!-- -->(.+?)<!-- --> <!-- -->(.+?)<!-- --> <!-- -->minute', str(container))
    if n:
        minute = n.group(2)
    runtime_minutes = int(hour)*60 + int(minute)
    data.append(runtime_minutes)

# create a new dataframe "imdb_top_movies", with a separate column for every extracted information
# the respective information is addressed by iterating through the previously created list "data", in steps of 13
imdb_top_movies = pd.DataFrame()
imdb_top_movies["IMDb id"] = data[0::13]
imdb_top_movies["Movie name"] = data[1::13]
imdb_top_movies["Year"] = data[2::13]
imdb_top_movies["Director"] = data[3::13]
imdb_top_movies["Stars"] = data[4::13]
imdb_top_movies["Rating"] = data[5::13] 
imdb_top_movies["Number of reviews"] = data[6::13]
imdb_top_movies["Genre"] = data[7::13]
imdb_top_movies["Country"] = data[8::13]
imdb_top_movies["Language"] = data[9::13]
imdb_top_movies["Budget"] = data[10::13]
imdb_top_movies["Gross box Office Revenue"] = data[11::13]
imdb_top_movies["Runtime (in minutes)"] = data[12::13]

# finally save the dataframe to a new csv-file
imdb_top_movies.to_csv("imdb_top_movies.csv")

**Question 2.2** (5 points) If you group the movies by release years, show the number of movies at each decade in descending order.

In [3]:
# load the previously stored csv-file
imdb_top_movies = pd.read_csv("imdb_top_movies.csv", index_col = 0)

# define a column, showing the decade for each year
imdb_top_movies["Decade"] = (imdb_top_movies["Year"] // 10 * 10).astype(str) + " - " + ((imdb_top_movies["Year"] // 10 * 10) + 10).astype(str)

# count the number of movies per decade and print in descending order
movies_per_decade = imdb_top_movies.groupby(imdb_top_movies["Decade"])["Year"].count().reset_index(name = "Count").rename(columns = {"Year": "Decade"})
display(movies_per_decade.sort_values("Count", ascending = False))

,Decade,Count
8,2000 - 2010,48
9,2010 - 2020,43
7,1990 - 2000,41
6,1980 - 1990,27
3,1950 - 1960,23
5,1970 - 1980,19
4,1960 - 1970,18
2,1940 - 1950,12
1,1930 - 1940,7
0,1920 - 1930,6


**Quesion 2.3** (5 points) Show the number of movies by the distribution of runtime at quartile (0-25%, 25-50%, 50-75%, 75-100%).

In [7]:
# create a new variable that cuts the runtime column into 4 equally sized bins
quartile = pd.cut(imdb_top_movies["Runtime (in minutes)"], 4, labels = ["0-25%", "25-50%", "50-75%", "75-100%"])

# group data by these 4 bins and count the number of movies occuring in each bin
display(imdb_top_movies.groupby(quartile)["IMDb id"].count().reset_index(name = "Count").rename(columns = {"Runtime (in minutes)": "Quartile"}))


,Quartile,Count
0,"(66.829, 109.75]",64
1,"(109.75, 152.5]",135
2,"(152.5, 195.25]",44
3,"(195.25, 238.0]",7


**Question 2.4** (5 points) Show the top 10 most popular actor/actresses in terms of number of movies they have starred. 

In [81]:
from collections import Counter

# create a list from the "Stars" column
stars_list = list(imdb_top_movies["Stars"])

# split each element of the list, so that every actor/actress builds a separate element in a sub-list
stars_list = [item.split(', ') for item in stars_list]

# resolve sub-list, so that only one big list of all actors/actresses remains
stars_list = [item for element in stars_list for item in element]

# write a function, that counts the occurrence of each actor/actress
def count_element(x):
    dictionary = {}
    for i in x:
        if i in dictionary:
            dictionary[i] += 1
        else:
            dictionary[i] = 1
    return dictionary

# apply this function to the list and print the top 10 actors/actresses with most occurrences
occurrences = count_element(stars_list)
actor_dict = dict(Counter(occurrences).most_common(10))
pd.DataFrame(actor_dict.items(), columns=['Stars', 'Count'])

,Stars,Count
0,Robert De Niro,9
1,Leonardo DiCaprio,6
2,Harrison Ford,6
3,Christian Bale,5
4,Clint Eastwood,5
5,Charles Chaplin,5
6,Morgan Freeman,4
7,Al Pacino,4
8,Tom Hanks,4
9,Brad Pitt,4


**Question 2.5** (5 points) Show the average ratings of movies across the genres and decades.

In [89]:
### Because I wasn't exactly sure, whether you want us to calculate it in a pivot table or separately, I did both

#1 pivot table

# extract only the columns "Genre" and "Rating" from "imdb_top_movies" dataframe and store them in new variable
df = imdb_top_movies[["Genre", "Rating"]]

# split the columns with multiple genres and create a new row for each of them and set the rating column as index 
new_df = pd.DataFrame(df.Genre.str.split(', ').tolist(), index = df.Rating).stack()

# remove the secondary index
new_df = new_df.reset_index([0, 'Rating'])

# rename the columns
new_df.columns = ['Rating', 'Genre']

# define the decade column again for "new_df"
new_df["Decade"] = (imdb_top_movies["Year"] // 10 * 10).astype(str) + " - " + ((imdb_top_movies["Year"] // 10 * 10) + 10).astype(str)

# create a pivot table, showing the average rating across genres (as rows) and decades (as columns)
display(round(new_df.pivot_table(index = "Genre", columns = "Decade", values = "Rating", aggfunc = "mean"), 1))

#2 separate calculation

# show the average rating per decade
avg_rating = pd.DataFrame(imdb_top_movies.groupby("Decade")["Rating"].mean().reset_index(name = "Avg_Rating").rename(columns = {"Year": "Decade"}))
print("\nAverage rating per decade:")
display(avg_rating.sort_values("Avg_Rating", ascending = False))

# show the average rating per genre
avg_rating_genre = pd.DataFrame(new_df.groupby("Genre")["Rating"].mean().reset_index()).rename(columns={'Rating':'Avg_Rating'})
print("\nAverage rating per genre:")
display(avg_rating_genre.sort_values("Avg_Rating", ascending = False))

Decade,1920 - 1930,1930 - 1940,1940 - 1950,1950 - 1960,1960 - 1970,1970 - 1980,1980 - 1990,1990 - 2000,2000 - 2010,2010 - 2020,2020 - 2030
Genre,,,,,,,,,,,
Action,NaN,8.3,NaN,8.4,8.6,8.6,8.8,8.7,8.6,8.5,NaN
Adventure,8.4,NaN,NaN,8.6,NaN,8.6,8.8,8.5,8.5,8.4,NaN
Animation,NaN,NaN,NaN,8.4,8.5,NaN,8.4,8.4,8.5,8.4,NaN
Biography,NaN,NaN,NaN,NaN,8.5,NaN,NaN,8.6,NaN,8.4,NaN
Comedy,NaN,8.4,8.5,8.3,8.5,NaN,8.4,8.5,8.5,8.4,8.3
Crime,NaN,8.6,NaN,8.6,8.4,8.7,NaN,8.6,8.7,8.6,NaN
Drama,NaN,8.5,8.5,8.4,8.6,8.5,8.5,8.8,8.5,8.5,NaN
Family,NaN,NaN,NaN,NaN,8.6,NaN,NaN,NaN,8.4,8.6,NaN
Fantasy,NaN,NaN,NaN,8.6,8.6,NaN,8.6,8.5,NaN,8.4,NaN



Average rating per decade:


,Decade,Avg_Rating
7,1990 - 2000,8.404878
10,2020 - 2030,8.383333
5,1970 - 1980,8.342105
4,1960 - 1970,8.316667
8,2000 - 2010,8.304167
3,1950 - 1960,8.291304
6,1980 - 1990,8.285185
2,1940 - 1950,8.266667
1,1930 - 1940,8.257143
9,2010 - 2020,8.255814



Average rating per genre:


,Genre,Avg_Rating
12,Music,8.400000
20,Western,8.383333
5,Crime,8.362745
16,Sci-Fi,8.360000
0,Action,8.350980
11,Horror,8.340000
14,Mystery,8.329032
6,Drama,8.325000
8,Fantasy,8.314286
1,Adventure,8.305000


**Question 2.6** (10 points) Now let's retrieve data from Bechdel Test Movie website [for each movie](https://bechdeltest.com/). You can send the requests to the API: https://bechdeltest.com/api/v1/doc#getMovieByImdbId. For example, for the movie The Shawshank Redemption (the IMDb id: 0111161), you can simply call: http://bechdeltest.com/api/v1/getMovieByImdbId?imdbid=0111161. 

Create a dataframe ```bechdel_imdb_top``` that merge the bechdel test info with the ```imdb_top_movies``` show how many top 250 movies are also in the bechdel test website.

In [91]:
# retrieve all title-URL tags of the top 250 movies again
page = requests.get("https://www.imdb.com/chart/top/")
soup = BeautifulSoup(page.content, 'html.parser')
title_urls = soup.find_all(class_="titleColumn")

# loop over all instances, extract the IMDb id of each movie and concatenate it with the API-URL-path
sub_pages = []
for titles in title_urls:
    find_url = titles.find("a", href = True)
    url = find_url["href"]
    url = url.replace("/title/tt", "http://bechdeltest.com/api/v1/getMovieByImdbId?imdbid=")
    url = url[:-1]
    sub_pages.append(url)

# loop over every API, extract the JSON-data and gather it in a list object
df = []
for i in sub_pages:
    response = requests.get(i)
    data = pd.read_json(response.text, lines = True)
    df.append(data)

# concatenate all list elements in a dataframe
bechdel_df = pd.concat(df, ignore_index = True)

# rename the key column, so that it matches the key column of the "imdb_top_movies" dataframe
bechdel_df.rename(columns={'imdbid':'IMDb id'}, inplace = True)

# changing "imdb_top_movies" key column to int, to make it the same as in "bechdel_df" and enable a merge
imdb_top_movies["IMDb id"] = imdb_top_movies["IMDb id"].astype(int)

# execute an inner merge of the 2 dataframes on "IMDb id" column and store it in new dataframe "bechdel_imdb_top"
bechdel_imdb_top = pd.merge(bechdel_df, imdb_top_movies, on = "IMDb id", how = "inner")

# print the result
print("{0} of the top 250 movies are also in the bechdel test website.".format(len(bechdel_imdb_top)))

# save the merged dataframe in a csv-file to be able to access it quicker in further operations
bechdel_imdb_top.to_csv("bechdel_imdb_top.csv")

241 of the top 250 movies are also in the bechdel test website.


**Question 2.7** (5 points) Show the percenage of movies given differen genres that has passed the test in different ways (Number from 0 to 3 (0 means no two women, 1 means no talking, 2 means talking about a man, 3 means it passes the test)) List the top 5 genres that has the highest fail rate (i.e., the movie passes 0 test)

In [98]:
# read the previously saved csv-file with bechdel and top 250 movies data
bechdel = pd.read_csv("bechdel_imdb_top.csv")

# split the genres column again, as in Question 2.5
new_df = pd.DataFrame(bechdel.Genre.str.split(', ').tolist(), index = bechdel.rating).stack()

# remove the secondary index
new_df = new_df.reset_index([0, 'rating'])

# rename the columns
new_df.columns = ['rating', 'Genre']

# count the number of movies per genre and rating, as well as the number of movies only per genre and store them in 2 variables 
bechdel_genre = new_df.groupby(["Genre", "rating"])["rating"].count().unstack()
tests_per_genre = new_df.groupby(["Genre"])["rating"].count()

# calculate the percentual share of every rating result per genre and overwrite the asbolute numbers
# round the results to one decimal place, fill NaN with 0 and transform columns to string, to add a percentage sign
bechdel_genre[0] = (bechdel_genre[0] / tests_per_genre) * 100
bechdel_genre[0] = bechdel_genre[0].round(1).fillna(0).astype(str) + " %"
bechdel_genre[1] = (bechdel_genre[1] / tests_per_genre) * 100
bechdel_genre[1] = bechdel_genre[1].round(1).fillna(0).astype(str) + " %"
bechdel_genre[2] = (bechdel_genre[2] / tests_per_genre) *100
bechdel_genre[2] = bechdel_genre[2].round(1).fillna(0).astype(str) + " %"
bechdel_genre[3] = (bechdel_genre[3] / tests_per_genre) *100
bechdel_genre[3] = bechdel_genre[3].round(1).fillna(0).astype(str) + " %"

# print the results and set NaN to 0
print("Percentage of the movies' test results per genre:")
display(pd.DataFrame(bechdel_genre))

# reformat the 0 column from string back to float, in a new column "sort"
bechdel_genre["sort"] = bechdel_genre[0].str.extract("(\d+.\d+)", expand = False).astype(float)

# use this column to sort the values
bechdel_genre.sort_values("sort", inplace = True, ascending = False)

# get rid of the column again
bechdel_genre = bechdel_genre.drop("sort", axis = 1)

# print the results
print("\nTop 5 genres with highest fail rate:")
display(pd.DataFrame(bechdel_genre[0].head(5)))

Percentage of the movies' test results per genre:


rating,0.0,1.0,2.0,3.0
Genre,,,,
Action,19.6 %,29.4 %,2.0 %,49.0 %
Adventure,21.7 %,33.3 %,3.3 %,41.7 %
Animation,13.0 %,43.5 %,0.0 %,43.5 %
Biography,14.8 %,37.0 %,14.8 %,33.3 %
Comedy,21.7 %,34.8 %,8.7 %,34.8 %
Crime,16.3 %,36.7 %,16.3 %,30.6 %
Drama,20.4 %,32.9 %,12.6 %,34.1 %
Family,10.0 %,20.0 %,0.0 %,70.0 %
Fantasy,7.1 %,28.6 %,14.3 %,50.0 %



Top 5 genres with highest fail rate:


,0.0
Genre,
War,50.0 %
Western,33.3 %
Romance,30.4 %
Film-Noir,25.0 %
History,25.0 %


**Question 2.8** (5 points) Show the top 10 highest-rated English and non-English movies that passed the test completely (rating=3), respectively. 

In [101]:
# create new dataframe, only showing the movies, which passed the bechdel test completely
bechdel_passed = bechdel[bechdel["rating"] == 3]

# from this create 2 more dataframes, one where "English" appears in the "Language" column and one where not
bechdel_english = bechdel_passed[bechdel_passed['Language'].str.contains('English')]
bechdel_non_english = bechdel_passed[~bechdel_passed['Language'].str.contains('English')]

# break the 2 dataframes down to the relevant 3 columns
bechdel_english = bechdel_english[["Movie name", "Language", "Rating"]]
bechdel_non_english = bechdel_non_english[["Movie name", "Language", "Rating"]]

# print the results
print("Top 10 highest-rated English movies:")
display(bechdel_english.sort_values("Rating", ascending = False).head(10))
print("\nTop 10 highest-rated non-English movies:")
display(bechdel_non_english.sort_values("Rating", ascending = False).head(10))

Top 10 highest-rated English movies:


,Movie name,Language,Rating
2,The Dark Knight,"English, Mandarin",9.0
5,Schindlers Liste,"English, Hebrew, German, Polish, Latin",9.0
7,Pulp Fiction,"English, Spanish, French",8.9
12,Inception,"English, Japanese, French",8.8
13,Der Herr der Ringe: Die zwei Türme,"English, Sindarin, Old English",8.8
15,Matrix,English,8.7
16,GoodFellas - Drei Jahrzehnte in der Mafia,"English, Italian",8.7
20,Ist das Leben nicht schön?,"English, French",8.6
21,Das Schweigen der Lämmer,"English, Latin",8.6
25,Interstellar,English,8.6



Top 10 highest-rated non-English movies:


,Movie name,Language,Rating
79,Prinzessin Mononoke,Japanese,8.4
82,Your Name.,Japanese,8.4
91,Capernaum: Stadt der Hoffnung,"Arabic, Amharic",8.4
95,M: Eine Stadt sucht einen Mörder,German,8.3
115,Nader und Simin - eine Trennung,Persian,8.3
205,Ballade vom Weg,Bengali,8.3
132,Pans Labyrinth,Spanish,8.2
158,Das wandelnde Schloss,Japanese,8.2
166,Mein Nachbar Totoro,Japanese,8.1
190,Wild Tales: Jeder dreht mal durch!,Spanish,8.1


**Question 2.9** (5 points) Creat a new column ROI that measures the return of investment using (box revenue-budget)/budget, and compare the average ROI between movies that passed (rating=3) and failed the test (rating=0), are their ROI different? Explain.

In [104]:
# read "bechdel_imdb_top" dataframe again
bechdel = pd.read_csv("bechdel_imdb_top.csv")

# remove all rows, where budget and box revenue are not in the same currency
bechdel.drop(bechdel[bechdel['Budget'].str[0] != bechdel["Gross box Office Revenue"].str[0]].index, inplace = True)

# get rid of all characters except numbers in the 2 columns
bechdel[['Budget', "Gross box Office Revenue"]] = bechdel[['Budget', "Gross box Office Revenue"]].replace(regex=[r'\D+'], value="").astype(float) 

# calculate the ROI and display it in a new column
bechdel["ROI (in %)"] = (bechdel["Gross box Office Revenue"] - bechdel["Budget"]) / bechdel["Budget"] * 100

# break down the dataframe to only ratings of 3 or 0
bechdel_0_3 = bechdel[(bechdel["rating"] == 0) | (bechdel["rating"] == 3)]

# calculate the average ROI for every possible rating and print the result
avg_ROI = pd.DataFrame(bechdel_0_3.groupby(bechdel_0_3["rating"])["ROI (in %)"].mean())
display(avg_ROI)
print("\nAnswer: Yes, the ROI of movies that passed and movies that failed are very different. For the movies that passed it's almost \ntwice the amount of the ROI of movies that failed. Probably that is because movies that passed the bechdel test are aiming at a greater target group, as they might be more likely to address women, for example.")

,ROI (in %)
rating,
0.0,587.984002
3.0,1008.182006



Answer: Yes, the ROI of movies that passed and movies that failed are very different. For the movies that passed it's almost 
twice the amount of the ROI of movies that failed. Probably that is because movies that passed the bechdel test are aiming at a greater target group, as they might be more likely to address women, for example.


**Question 2.10** (10 points) Now load the bech test results from all movies (https://bechdeltest.com/api/v1/doc#getAllMovies) that contains the all movies that are rated by the Bechdel Test website. Are women representation improved over the decades? Create a dataframe ```bechdel_imdb```, comparing the top 250 and other movies, in terms of percentage, how many passed/failed the test? 

In [116]:
# request all movies and store them in a dataframe
all_movies = pd.read_json(requests.get("https://bechdeltest.com/api/v1/getAllMovies").text)

# define a variable for each decade
decade = (all_movies["year"] // 10 * 10).astype(str) + " - " + ((all_movies["year"] // 10 * 10) + 10).astype(str)

# use the variable to group all movies and find the average bechdel rating for each group
avg_per_decade = pd.DataFrame(all_movies.groupby(decade)["rating"].mean()).rename(columns = {"rating": "avg_rating"})

# print the results
print("Average bechdel rating per decade:")
display(avg_per_decade.sort_values("avg_rating", ascending = False))
print("Answer: As you can see, the representation of women has definitely increased over the decades. The 4 strongest decades in terms of the bechdel rating have been the most recent 4 decades, in ascending order towards now.")

# rename columns of "all_movies" dataframe, to enable a merge with the top 250 movies dataframe
all_movies.rename(columns={'imdbid':'IMDb id'}, inplace = True)

# convert both "IMDb_id" columns to the same type and remove leading zeros in the "all_movies" dataframe 
all_movies["IMDb id"] = all_movies["IMDb id"].astype(str)
all_movies["IMDb id"] = all_movies["IMDb id"].str.lstrip('0')
imdb_top_movies["IMDb id"] = imdb_top_movies["IMDb id"].astype(str)

# finally conduct a left-merge of the 2 dataframes
bechdel_imdb = pd.merge(all_movies, imdb_top_movies, how = "left", on = "IMDb id")

# create 2 sub-dataframes (every movie which has NaN in the "Rating" column, is not in the top 250 dataframe and vice versa)
bechdel_top250 = bechdel_imdb[bechdel_imdb["Rating"].notna()]
bechdel_other = bechdel_imdb[bechdel_imdb["Rating"].isna()]

# count the number of top 250 movies that passed and failed the bechdel test 
passed = 0
failed = 0
for i in bechdel_top250["rating"]:
    if i == 3:
        passed += 1
    else:
        failed += 1

# print the results
print("\nNumber of top 250 movies, that passed the test: {0} %".format(round((passed / len(bechdel_top250) * 100), 2)))
print("Number of top 250 movies, that failed the test: {0} %".format(round((failed / len(bechdel_top250) * 100), 2)))

# count the number of other movies that passed and failed the bechdel test 
passed = 0
failed = 0
for i in bechdel_other["rating"]:
    if i == 3:
        passed += 1
    else:
        failed += 1

# print the results
print("\nNumber of other movies, that passed the test: {0} %".format(round((passed / len(bechdel_other) * 100), 2)))
print("Number of other movies, that failed the test: {0} %".format(round((failed / len(bechdel_other) * 100), 2)))

Average bechdel rating per decade:


,avg_rating
year,
2020 - 2030,2.426593
2010 - 2020,2.279810
2000 - 2010,2.201307
1990 - 2000,2.127224
1930 - 1940,2.073892
1980 - 1990,2.000000
1950 - 1960,1.993174
1940 - 1950,1.987552
1970 - 1980,1.895692


Answer: As you can see, the representation of women has definitely increased over the decades. The 4 strongest decades in terms of the bechdel rating have been the most recent 4 decades, in ascending order towards now.

Number of top 250 movies, that passed the test: 38.59 %
Number of top 250 movies, that failed the test: 61.41 %

Number of other movies, that passed the test: 57.47 %
Number of other movies, that failed the test: 42.53 %
